In [8]:
%load_ext autoreload

In [9]:
import os, sys, importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf

BASE_PATH = os.path.join(os.getcwd(), "..", "..")
MODULE_PATH = os.path.join(BASE_PATH, "modules")
DATASETS_PATH = os.path.join(BASE_PATH, "datasets")

sys.path.append(MODULE_PATH)

from data import BenchmarkData, DataSetType
from models import setup_growth, default_model
from bayesian import McDropout, MomentPropagation

In [10]:
setup_growth()

1 Physical GPU's,  1 Logical GPU's


In [11]:
benchmark = BenchmarkData(DataSetType.MNIST, os.path.join(DATASETS_PATH, "mnist"), classes=2)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(benchmark.inputs, benchmark.targets)

In [13]:
print("Datapoints: {}".format(len(benchmark.targets)))
print("Unique labels: {}".format(np.unique(benchmark.targets)))

Datapoints: 14780
Unique labels: [0 1]


In [14]:
model = default_model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["binary_accuracy"])

In [15]:
model.evaluate(x_test, y_test)

116/116 [==============================] - 2s 16ms/step - loss: 4.1283 - binary_accuracy: 0.4736


[4.128287315368652, 0.4736129939556122]

In [16]:
%autoreload 2
dp_model = McDropout(model)

In [18]:
%autoreload 2
dp_model.evaluate(x_test, y_test)

[2.796782698233213, 0.5372124492557511]

In [20]:
dp_model.fit(x_train, y_train, batch_size=60)

185/185 [==============================] - 5s 28ms/step - loss: 0.3102 - binary_accuracy: 0.9840


In [22]:
dp_model.evaluate(x_test, y_test)

[0.7615203117972562, 0.48660351826792964]

In [23]:
dp_model.fit(x_train, y_train, batch_size=60)

185/185 [==============================] - 1s 5ms/step - loss: 0.0045 - binary_accuracy: 0.9986


In [24]:
dp_model.evaluate(x_test, y_test)

[0.7671364879128623, 0.49039242219215157]